# Case Study: Scraping Reddit with Pushshift

This lesson will walk you through the basics of scraping Reddit data using the popular Pushshift API.  We'll discuss some of the most useful parameters at our disposal, then work our way through a number of exercises putting those parameters to use.  Finally, we'll have some fun visualizing patterns in Reddit content. 

## Reddit

[Reddit](https://www.reddit.com/) provides an additional avenue for large-scale digital data collection and analysis.


Notably, Reddit allows its users, known as Redditors, to create and maintain communities, or "subreddits," within the larger Reddit platform.  Redditors can opt in or out of a wide variety of communities at any time, allowing them to personalize their feeds and engage with a uniquely tailored online social circle. 

## Pushshift

Per the [subreddit](https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/) dedicated to its discussion, pushshift.io allows users to analyze and aggregate large volumes of data from Reddit, while also providing the option to specify the desired ranges time from which to collect data.  


### Setup

As in the previous chapter discussing more general web scraping through APIs, we'll be importing both `requests` and `pandas`. 

In [1]:
import requests
import pandas as pd

We'll also want to save our base URL as a string for later use.  When using Pushshift, we'll have to indicate whether we want to scrape submissions or comments in our request within the base URL. To scrape submissions, the base URL should end with `/submission/`. To scrape comments, the base URL should end with `/comment/`.  So, if we want to request submissions we'd set our base URL as the following: 

In [17]:
base_url = 'https://api.pushshift.io/reddit/search/submission/'

### Parameters

We'll now discuss a number of the parameters available through Pushshift. You can find a full list of parameters available through the API [here](https://pushshift.io/api-parameters/).  We'll return to some of the parameters listed at the link provided later on in the chapter, when we begin analyzing some scraped Reddit content. 

- **sort**: Allows you to control the direction in which you scrape results. Results can be scraped in ascending ('asc') or descending ('desc') order. 


> - **sort_type**: Allows you to set the parameter used to sort requests. 


- **created_utc**: This parameter is useful for restricting requests by the Coordinated Universal Time, or UTC, of their creation.  There are a number of online conversion tools available for converting human-readable dates to UTCs; I typically use [this one](https://www.epochconverter.com/).


> - **before**: Use this to scrape content created *before* a certain UTC.
- **after**: Use this to scrape content created *after* a certain UTC.


- **size**: Allows you to set a maximum size limit on your request.  Without looping requests, size per request maxes out at 1,000 submissions or comments.


- **author**: Allows you to request only the content produced by a single Redditor, or a specific set of Redditors.


- **subreddit**: Allows you to request only the content from a single subreddit, or a specific set of subreddits.


- **score**: Allows you to sort or otherwise restrict requests depending on the net number of likes versus dislikes received by content.


- **num_comments**: Allows you to sort or otherwise restrict requests depending on the total number of comments on submissions. 


### Pushshift Exercise 1: Making a Request

Using the API Documentation outlined above as a guide, let's use pushshift to scrape some Reddit content.  We'll start by looking at some submissions from [r/changemyview](https://www.reddit.com/r/changemyview/), which describes itself as "a place to post an opinion you accept may be flawed, in an effort to understand other perspectives on the issue."


Let's look at the 50 submissions to r/changemyview from 2019 that received the highest number of comments. We can start by setting the parameters for our request in a dictionary: 

In [14]:
parameters = {'subreddit' : 'changemyview',
              'sort'       : 'desc',
              'sort_type'  : 'num_comments',
              'after'       : '1546300800',
              'before'       : '1577836800',
              'size'       : 50,}     

Now we're ready to make our request to the API.

In [15]:
r = requests.get(base_url, params = parameters)

We'll want to store the data scraped in the request in JSON format, so we can access it within the notebook as a dataframe. 

In [11]:
df = pd.DataFrame(r.json()['data'])

Try looking at a random sample of 10 submissions from the request as a dataframe in the cell below.

In [19]:
df.sample(10)

,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_patreon_flair,can_mod_post,...,allow_live_comments,awarders,steward_reports,total_awards_received,author_premium,og_description,og_title,gilded,post_hint,preview
26,kinglax,NaN,None,[],None,NaN,text,t2_5lo3i,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
13,GAMpro,NaN,None,[],None,NaN,text,t2_ysnrw,False,False,...,False,[],[],0.0,NaN,NaN,NaN,NaN,self,"{'enabled': False, 'images': [{'id': 'C9KSOp_b..."
48,juul_pod,NaN,None,[],None,NaN,text,t2_174yan,False,False,...,False,[],[],0.0,False,NaN,NaN,NaN,NaN,NaN
5,RedUlster,NaN,None,[],None,NaN,text,t2_4hnldl0n,False,False,...,False,[],[],0.0,NaN,,,NaN,NaN,NaN
47,gucci_sweatbands,NaN,None,[],None,NaN,text,t2_1m2qo0k9,False,False,...,False,NaN,NaN,0.0,NaN,NaN,NaN,2.0,NaN,NaN
45,lawtonj,,None,[],7∆,dark,text,t2_n3v7d,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
34,Hey-I-Read-It,NaN,None,[],None,NaN,text,t2_23pcrwbr,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Highlyemployable,NaN,None,[],None,NaN,text,t2_4p7iimbi,False,False,...,False,[],[],0.0,NaN,,,1.0,NaN,NaN
49,Caesaroctopus,NaN,None,[],None,NaN,text,t2_prc43,False,False,...,False,[],[],0.0,False,NaN,NaN,NaN,NaN,NaN
33,Cadent_Knave,NaN,None,[],None,NaN,text,t2_iz35j,False,False,...,False,[],[],0.0,False,NaN,NaN,NaN,NaN,NaN


There's a lot of information available!  It will be useful to determine a list of keys in the JSON file we'd like to look at, so we're not bombarded with too much information at one time.   

In [12]:
df.keys()

Index(['author', 'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_text', 'author_flair_text_color',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'can_mod_post', 'contest_mode', 'created_utc', 'domain', 'full_link',
       'gildings', 'id', 'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'suggested_sort', 'thumbnail', 'title', 'updated_utc',
       'url', 'whitelist_status', 'wls', 'all_awa

Some of these keys (for example,`author`, `created_utc`, `num_comments`, and `score`) were outlined in the API definitions above, but a few of the keys are new.  Here are a couple of important keys we haven't yet discussed:

- **title**: This is the title a Redditor has given their submission.  


- **selftext**: This is the text contained within the submission itself. While a submission will always contain some text in the `title`, some image-only submissions will not contain any text in `selftext`. 

To get a better handle of the data we've scraped, let's narrow our focus to `author`, `num_comments`, `score`, `title`, and `selftext`.

In [27]:
columns_to_keep = ['author', 'num_comments', 'score', 'title', 'selftext']

Now, let's pull another random sample of 10 submissions from the dataframe using our narrowed set of parameters. 

In [9]:
df[columns_to_keep].sample(10)

,author,num_comments,score,title,selftext
15,Birb-Brain-Syn,1468,3491,CMV: Anyone working a full-time job should be ...,So for me the above held opinion is straight-f...
0,Amiller1776,2617,1337,CMV: Trans activists who claim it is transphob...,Several trans activist youtubers have posted v...
42,BasicRedditor1997,1100,1,CMV:Making children pledge allegiance to the f...,Every day from Kindergarten through 12th grade...
41,mandi4910,1112,3046,CMV: Transgender women shouldn't be allowed to...,"Ok first of all i really, really feel like a j..."
30,psychoIogic,1254,1696,CMV: Circumcision of babies should be illegal ...,Some people believe there are benefits that wa...
9,Akukurotenshi,1783,1847,CMV: Eating dogs is not wrong,You always see people demeaning those who have...
20,Insert_Alias_Heree,1377,2274,"CMV: Those born in poor economic backgrounds, ...",With the increasing amount of Republican actio...
11,killgoresalmonman,1660,3262,CMV: There’s no fact or logic based reason to ...,The science is clear that climate change is re...
12,ThrowingThisAwayBtw,1594,1,CMV: Killing animals for pleasure is wrong,Hi all!\n\nI've recently been thinking a lot a...
6,carlsaganheaven,1877,3062,CMV: In heterosexual relationships the problem...,It's a common conception that when you marry a...


That's a bit easier to handle.  We can always add or take away keys as needed. 

For relatively simple (but easily obtainable) data visualizations through the Pushift API, we can check out the [Pushshift Reddit Search](https://redditsearch.io/)

### Pushshift Exercise 2